# Manipulação de Arquivos e Strings

### Semana 13 | Uso do Regex

## Questão Dirigida

Como gravar em um arquivo CSV os dados da pandemia de COVID-19 por país desta [página](https://pt.wikipedia.org/wiki/Portal:COVID-19) da Wikipedia?

## Desenvolvimento do Projeto

Para desenvolver este projeto, vamos utilizar técnicas de raspagem de dados na Web (Web Scraping), com a biblioteca `requests`, `BeautifulSoup` e expressões regulares.

In [ ]:
# download
import requests

# parsing do HTML
from bs4 import BeautifulSoup

# expressões regulares
import re

A incidência de COVID-19 por país pode ser encontrada na página da Wikipedia.

In [ ]:
# URL de análise
url_pagina = 'https://pt.wikipedia.org/wiki/Portal:COVID-19'

In [ ]:
# Utiliza a biblioteca requests para fazer o download da pagina HTML
pagina_html = requests.get(url_pagina)

# O Código 200 indica que o download foi bem sucedido
pagina_html

<Response [200]>

In [ ]:
# O Beautiful Soup 
soup = BeautifulSoup(pagina_html.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="pt">
 <head>
  <meta charset="utf-8"/>
  <title>
   Portal:COVID-19 – Wikipédia, a enciclopédia livre
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgRequestId":"8b69c4d3-37ef-4855-bd88-b02590485a56","wgCSPNonce":!1,"wgCanonicalNamespace":"Portal","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":100,"wgPageName":"Portal:COVID-19","wgTitle":"COVID-19","wgCurRevisionId":58084687,"wgRevisionId":58084687,"wgArticleId":6223270,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["!Páginas com argumentos formatnum não numéricos","!CS1 francês-fontes em língua (fr)","!CS1 usa script na língua 

**Inspeção da página**

Inspecionando o código-fonte da página HTML no navegador (Chrome, Firefox etc) podemos ver que a tabela de interesse contém a classe CSS: `wikitable sortable mw-collapsible no-ref`. Vamos utilizar esta classe CSS para restringir a busca pelos elementos de interesse.

In [ ]:
tb_covid_pais = soup.find(class_='wikitable sortable mw-collapsible no-ref')
#tb_covid_pais

Opcionalmente, pode-se fazer o seguinte.

In [ ]:
tbs_covid_pais = soup.find_all(class_='wikitable sortable mw-collapsible no-ref')
# Lembre-se que o método find_all retorna uma lista. No caso deste exemplo, 
# só existe um elemento na lista
tb_covid_pais = tbs_covid_pais[0]
# tb_covid_pais

In [ ]:
# Retorna uma lista com todas as linhas da tabela
linhas_tb = tb_covid_pais.find_all('tr')
#linhas_tb

> **Atenção**: É sempre importante entender o tipo de uma variável.

In [ ]:
type(linhas_tb)

bs4.element.ResultSet

In [ ]:
type(linhas_tb[0])

bs4.element.Tag

**Investigando os resultados**

Primeiro país ocorre na posição 2.

In [ ]:
linhas_tb[2].get_text()

'\n Estados Unidos[a]\n33\xa0503\xa0141\n\n602\xa0805\n\n–\n\n[2]\n'

Último ocorre na posição 226.

In [ ]:
linhas_tb[226].get_text()

'\n Saba\n7\n\n0\n\n7\n\n[291]\n'

In [ ]:
ini_tb = 2
fim_tb = 226

Brincando um pouco com esses objetos podemos ver que eles contém os dados que desejamos. No entanto, esses dados precisem ser tratados. Por exemplo: podemos ver que o resultado a seguir começa e termina com `\n` (caractere de nova linha); existe um espaço antes do nome do país; existem vários `\n` e `\xa0` (*line feed*) no meio da linha. Além disso, existem notas associadas ao nome do país e ao final da linha. Vamos cuidar de cada um desses problemas.

In [ ]:
lin = linhas_tb[2].get_text()
lin

'\n Estados Unidos[a]\n33\xa0503\xa0141\n\n602\xa0805\n\n–\n\n[2]\n'

##### Removendo os `\n` e os espaços no começo e ao final da linha. 

O método `strip()` remove caracteres no começo e ao final da linha.

In [ ]:
lin

'\n Estados Unidos[a]\n33\xa0503\xa0141\n\n602\xa0805\n\n–\n\n[2]\n'

In [ ]:
lin.strip("\n \t")

'Estados Unidos[a]\n33\xa0503\xa0141\n\n602\xa0805\n\n–\n\n[2]'

Note que a variável `lin` não é alterada desta forma.

In [ ]:
lin

'\n Estados Unidos[a]\n33\xa0503\xa0141\n\n602\xa0805\n\n–\n\n[2]\n'

Para que a variável `lin` seja alterada, é necessário utilizar a atribuição.

In [ ]:
lin = lin.strip("\n ")
lin

'Estados Unidos[a]\n33\xa0503\xa0141\n\n602\xa0805\n\n–\n\n[2]'

##### Removendo os caracteres `\xa0` e `\n` no meio da string

Podemos utilizar o método `replace()` para substituir o caractere `\xa0` com a string vazia, para que o número de casos não esteja com espaços entre eles. Note que o caractere `\xa0` está em Unicode (que podem ser representados por hexadecimais), por isso necessitam do `u` antes do começo da string.

In [ ]:
lin = lin.replace(u"\xa0", u"")
lin

'Estados Unidos[a]\n33503141\n\n602805\n\n–\n\n[2]'

Como desejamos criar um arquivo CSV ao final do processo, vamos substituir um ou mais `\n` e `\t` que separa cada coluna da tabela por um `\t`.

In [ ]:
lin = re.sub("[\n|\t]+","\t", lin)
lin

'Estados Unidos[a]\t33503141\t602805\t–\t[2]'

##### Removendo as notas do nome do país e ao final da linha

Para remover as notas do nome do país, vamos utilizar a seguinte expressão regular: `r"\[[\d|a-z]+\]"`. A expressão pode ser lida da seguinte forma: selecione o texto de dígitos ou caracteres alfa-numéricos que estejam entre `[` e `]`. 

In [ ]:
lin

'Estados Unidos[a]\t33503141\t602805\t–\t[2]'

In [ ]:
lin = re.sub(r"\[[\d|a-z]+\]", "", lin)
lin

'Estados Unidos\t33503141\t602805\t–\t'

Remove-se finalmente o `\t` ao final da string.

In [ ]:
# Coluna Fontes
lin = lin.rstrip("\t")
lin

'Estados Unidos\t33503141\t602805\t–'

In [ ]:
lin = lin.replace("–", "0")
lin

'Estados Unidos\t33503141\t602805\t0'

### Criação de uma função de preparação da linha

Para se criar a função, vamos começar aplicando a sequência de passos para preparar a linha dos EUA e em seguida analisar se o que fizemos é suficiente para os outros casos.

In [ ]:
def prep_linha_v1(linha):
    # Remove os caracteres \n no começo e final da string
    # Em seguida, remove os espaços
    linha = linha.strip("\n ")
    linha = linha.replace(u"\xa0", u"")
    linha = re.sub("[\n|\t]+","\t", linha)
    linha = re.sub(r"\[[\d|a-z]+\]", "", linha)
    linha = linha.strip("\t")
    linha = re.sub(r"–", "0", linha)
    return linha

Como esperado, a função funciona para os EUA.

In [ ]:
eua = prep_linha_v1(linhas_tb[2].get_text())
eua

'Estados Unidos\t33503141\t602805\t0'

#### Preste atenção nos tipos de dados da entrada das funções.

Chamada com String.

In [ ]:
type(linhas_tb[2].get_text())

str

In [ ]:
prep_linha_v1(linhas_tb[2].get_text())

'Estados Unidos\t33503141\t602805\t0'

Chamada com `Tag`

In [ ]:
type(linhas_tb[2])

bs4.element.Tag

In [ ]:
#prep_linha_v1(linhas_tb[2])

#### Investigando outros casos

Listando as transformações nos países.

In [ ]:
for i in range(ini_tb,fim_tb+1):
    linha = linhas_tb[i].get_text()
    print(i, prep_linha_v1(linha))

2 Estados Unidos	33503141	602805	0
3 Índia	26752447	303720	23728011
4 Brasil	16907425	472531	15290500
5 França	5707683	110002	0
6 Turquia	5287980	48164	5160774
7 Rússia	5126437	123787	4736446
8 Reino Unido	4516892	127840	0
9 Itália	4230153	126472	3908312
10 Argentina	3938961	80867	3497437
11 Espanha	3697987	80196	0
12 Alemanha	3717890	89825	3533648
13 Colômbia	3518046	90890	3267100
14 Irã	2954309	80813	2522702
15 Polônia	2875136	74152	2645150
16 México	2432280	228754	1937977
17 Ucrânia	2214517	51182	2093228
18 Peru	1980391	186073	1931773
19 Chéquia	1663517	30159	1624721
20 África do Sul	1691491	56929	1574223
21 Países Baixos	1661520	17675	0
22 Indonésia	1 511 712	40 858	1 348 330
23 Canadá	1391174	25712	1340123
24 Chile	1427956	29937	1347676
25 Filipinas	1269478	21898	1188243
26 Iraque	1218524	16488	1133347
27 Romênia	1078863	30815	1042898
28 Suécia	1078062	14523	0
29 Bélgica	1069874	25019	0
30 Paquistão	932140	21265	863111
31 Portugal	852646	17034	811897
32 Israel	839539	6417	832923
3

Pela Grécia, vemos que remos ter tratar um comando HTML em uma das colunas.

In [ ]:
prep_linha_v1(linhas_tb[48].get_text())

'Grécia\t407857\t12218\tdata-sort-value="−1"'

A Ossétia do Sul também necessita ser tratada.

In [ ]:
prep_linha_v1(linhas_tb[183].get_text())

'Ossétia do Sul\t3346\t60+\t3198'

In [ ]:
def prep_linha_v2(linha):
    # Remove os caracteres \n no começo e final da string
    # Em seguida, remove os espaços
    linha = linha.strip("\n ")
    linha = linha.replace(u"\xa0", u"")
    linha = re.sub("[\n|\t]+","\t", linha)
    linha = re.sub(r"\[[\d|a-z]+\]", "", linha)
    linha = linha.strip("\t")
    linha = re.sub(r"–", "0", linha)
        
    # Trata a Grécia (parte por parte)
    linha = linha.replace("data-sort-value=\"−1\"", "0")
    
    # Trata a Ossétia do Sul
    linha = linha.replace("+", "")
    return linha

In [ ]:
prep_linha_v2(linhas_tb[48].get_text())

'Grécia\t407857\t12218\t0'

In [ ]:
prep_linha_v2(linhas_tb[183].get_text())

'Ossétia do Sul\t3346\t60\t3198'

Analisando a Indonésia, descobrimos mais uma surpresa.

In [ ]:
prep_linha_v2(linhas_tb[22].get_text())

'Indonésia\t1 511 712\t40 858\t1 348 330'

Para facilitar, vamos tratar este caso direto no Pandas.

##### Preparando o texto do arquivo final

Para criarmos o texto arquivo CSV final, basta fazer uma concatenação de todas as linhas da tabela, incluindo um `\n` ao final da linha. 

In [ ]:
# Esta primeira linha define o título da tabela
texto_final = "País\tCasos\tMortes\tCurados\n"
for i in range(ini_tb, fim_tb+1):
    # Preste atenção no .get_text(), que transforma a chamada em string
    linha = prep_linha_v2(linhas_tb[i].get_text())
    texto_final += linha
    if (i < fim_tb):
        texto_final += "\n"
print(texto_final)

País	Casos	Mortes	Curados
Estados Unidos	33503141	602805	0
Índia	26752447	303720	23728011
Brasil	16907425	472531	15290500
França	5707683	110002	0
Turquia	5287980	48164	5160774
Rússia	5126437	123787	4736446
Reino Unido	4516892	127840	0
Itália	4230153	126472	3908312
Argentina	3938961	80867	3497437
Espanha	3697987	80196	0
Alemanha	3717890	89825	3533648
Colômbia	3518046	90890	3267100
Irã	2954309	80813	2522702
Polônia	2875136	74152	2645150
México	2432280	228754	1937977
Ucrânia	2214517	51182	2093228
Peru	1980391	186073	1931773
Chéquia	1663517	30159	1624721
África do Sul	1691491	56929	1574223
Países Baixos	1661520	17675	0
Indonésia	1 511 712	40 858	1 348 330
Canadá	1391174	25712	1340123
Chile	1427956	29937	1347676
Filipinas	1269478	21898	1188243
Iraque	1218524	16488	1133347
Romênia	1078863	30815	1042898
Suécia	1078062	14523	0
Bélgica	1069874	25019	0
Paquistão	932140	21265	863111
Portugal	852646	17034	811897
Israel	839539	6417	832923
Hungria	805871	29842	714329
Bangladesh	809314	12801	749425
J

## Gravando o texto em um arquivo

Para se gravar o texto em um arquivo, basta utilizar o código a seguir.

In [ ]:
arquivo = open("lista-paises.tsv", "w")
arquivo.write(texto_final)
arquivo.close()

### Lendo o arquivo CSV com o Pandas

In [ ]:
import pandas as pd
df = pd.read_csv("lista-paises.tsv", sep='\t')
df

,País,Casos,Mortes,Curados
0,Estados Unidos,33503141,602805,0
1,Índia,26752447,303720,23728011
2,Brasil,16907425,472531,15290500
3,França,5707683,110002,0
4,Turquia,5287980,48164,5160774
...,...,...,...,...
220,Santo Eustáquio,20,0,20
221,Montserrat,20,1,18
222,MS Zaandam,13,4,0
223,Coral Princess,12,3,0


### Indonésia

In [ ]:
df[df['País'] == 'Indonésia']

,País,Casos,Mortes,Curados
20,Indonésia,1 511 712,40 858,1 348 330


A função lambda é uma função anônima (sem nome), em que o parâmetro é designado por celula. A função é aplicada sobre cada célula da coluna. Assim para cada célula, a função replace será executada.

In [ ]:
df['Casos'] = df.Casos.apply(lambda celula: celula.replace(" ", ""))
df['Mortes'] = df.Mortes.apply(lambda celula: celula.replace(" ", ""))
df['Curados'] = df.Curados.apply(lambda celula: celula.replace(" ", ""))

In [ ]:
df[df['País'] == 'Indonésia']

,País,Casos,Mortes,Curados
20,Indonésia,1511712,40858,1348330


**Convertendo para tipos numéricos**

Se tudo correr bem nessa conversão, é porque tudo foi tratado adequadamente. Caso ocorra problemas aqui, você deve inspecionar a tabela para encontrar onde estão os problemas.

In [ ]:
df['Casos'] = df.Casos.astype(int)
df['Mortes'] = df.Mortes.astype(int)
df['Curados'] = df.Curados.astype(int)

### Tarefas

In [ ]:
df

,País,Casos,Mortes,Curados
0,Estados Unidos,33503141,602805,0
1,Índia,26752447,303720,23728011
2,Brasil,16907425,472531,15290500
3,França,5707683,110002,0
4,Turquia,5287980,48164,5160774
...,...,...,...,...
220,Santo Eustáquio,20,0,20
221,Montserrat,20,1,18
222,MS Zaandam,13,4,0
223,Coral Princess,12,3,0


1) Utilizando o plotly, faça a análise da evolução de Casos, Mortes e Curados do Dataset. 

In [ ]:
import plotly.express as px

In [ ]:
fig1 = px.bar(df[:10], x="País", y="Casos", title="Os 10 países com mais casos de COVID-19")
fig1.show()

In [ ]:
df_mortes = df.sort_values(by="Mortes", ascending=False)
fig2 = px.bar(df_mortes[:10], x="País", y="Mortes", title="Os 10 países com mais mortes por COVID-19")
fig2.show()

In [ ]:
df_curados = df.sort_values(by="Curados", ascending=False)
fig3 = px.bar(df_curados[:10], x="País", y="Curados", title="Os 10 países com mais curados de COVID-19")
fig3.show()

In [ ]:
#excluindo países que apresentem número de casos, mortes ou curados == 0
df_dispersao = df.loc[(df.Mortes != 0) & (df.Curados != 0) & (df.Casos != 0)]
fig4 = px.scatter(df_dispersao, x="Casos", y="Mortes", title="Relação entre casos e mortes de COVID-19 por País", hover_name="País", color="País")
fig4.show()

2) Calcule a estatística descritiva (média, soma, mediana e desvio-padrão) dos países do dataset.

In [ ]:
df.Mortes.describe()

count       225.000000
mean      16362.488889
std       60765.311395
min           0.000000
25%          86.000000
50%         818.000000
75%        5448.000000
max      602805.000000
Name: Mortes, dtype: float64

In [ ]:
print("Estatísticas sobre o número de mortes:")
print(f"Média: {round(df.Mortes.describe()[1])} mortos")
print(f"Total de mortos: {df.Mortes.sum()} mortos")
print(f"Mediana do total de mortos por país: {df.Mortes.describe()[5]} mortos")
print(f"Desvio Padrão do total de mortos por país: {round(df.Mortes.describe()[2])} mortos")

Estatísticas sobre o número de mortes:
Média: 16362 mortos
Total de mortos: 3681560 mortos
Mediana do total de mortos por país: 818.0 mortos
Desvio Padrão do total de mortos por país: 60765 mortos


In [ ]:
print("Estatísticas sobre o número de casos:")
print(f"Média: {round(df.Casos.describe()[1])} casos")
print(f"Total de casos: {df.Casos.sum()} casos")
print(f"Mediana do total de casos por país: {df.Casos.describe()[5]} casos")
print(f"Desvio Padrão do total de casos por país: {round(df.Casos.describe()[2])} casos")

Estatísticas sobre o número de casos:
Média: 759434 casos
Total de casos: 170872610 casos
Mediana do total de casos por país: 48901.0 casos
Desvio Padrão do total de casos por país: 3161962 casos


3) Encontre uma tabela na Wikipedia do seu interesse e aplique as técnicas apresentadas neste caderno para criar um DataFrame, criar gráficos no Plotly e fazer as estatísticas descritivas.

In [64]:
import pandas as pd
from bs4 import BeautifulSoup
import plotly.express as px
import requests
import re

In [3]:
url = "https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory"
page2 = requests.get(url)
page2

<Response [200]>

In [38]:
soup2 = BeautifulSoup(page2.content, "html.parser")
table = soup2.find(attrs={"aria-label":"COVID-19 pandemic mortality and death rates by country table"}).table.tbody

In [42]:
table_lines = table.find_all("tr")
table_lines[1].text

'\n\xa0Peru\n1,980,391\n186,073\n9.40%\n572.35\n'

In [50]:
def consertar_linha(row):
  row = row.replace("\xa0", "")
  row = row.strip("\n")
  row = re.sub("\n+", "\t", row)
  return row

In [53]:
consertar_linha(table_lines[1].text)

'Peru\t1,980,391\t186,073\t9.40%\t572.35'

In [59]:
dataset = "País\tCasos Confirmados\tMortes\tTaxa de Fatalidade\tMortes por cem mil hab.\n"
for line in table_lines[1:]:
  dataset += consertar_linha(line.text)
  if line != table_lines[-1]:
    dataset += "\n"
print(dataset)

País	Casos Confirmados	Mortes	Taxa de Fatalidade	Mortes por cem mil hab.
Peru	1,980,391	186,073	9.40%	572.35
Hungary	806,008	29,854	3.70%	305.57
Bosnia and Herzegovina	204,304	9,374	4.60%	283.97
Czech Republic	1,663,517	30,159	1.80%	282.66
San Marino	5,090	90	1.80%	265.80
North Macedonia	155,417	5,455	3.50%	261.82
Montenegro	99,812	1,592	1.60%	255.89
Bulgaria	419,473	17,820	4.20%	255.46
Moldova	255,453	6,134	2.40%	230.81
Slovakia	390,436	12,404	3.20%	227.43
Brazil	16,947,062	473,404	2.80%	224.31
Belgium	1,070,802	25,033	2.30%	217.98
Slovenia	255,375	4,388	1.70%	210.16
Italy	4,232,428	126,523	3.00%	209.83
Croatia	357,565	8,086	2.30%	198.80
Poland	2,875,136	74,152	2.60%	195.29
United Kingdom	4,532,802	128,103	2.80%	191.67
Colombia	3,571,067	91,961	2.60%	182.68
United States	33,362,600	597,628	1.80%	182.07
Argentina	3,955,439	81,214	2.10%	180.72
Mexico	2,433,681	228,804	9.40%	179.35
Spain	3,697,981	80,196	2.20%	170.35
Portugal	852,646	17,034	2.00%	165.87
Andorra	13,758	127	0.90%	164.63
Fr

In [62]:
arquivo2 = open("Dados_covid_relativos.tsv", "w")
arquivo2.write(dataset)
arquivo2.close()

In [68]:
df_new = pd.read_csv("Dados_covid_relativos.tsv", sep="\t").sort_values(by="Mortes por cem mil hab.", ascending=False)
df_new

,País,Casos Confirmados,Mortes,Taxa de Fatalidade,Mortes por cem mil hab.
0,Peru,"1,980,391","186,073",9.40%,572.35
1,Hungary,"806,008","29,854",3.70%,305.57
2,Bosnia and Herzegovina,"204,304","9,374",4.60%,283.97
3,Czech Republic,"1,663,517","30,159",1.80%,282.66
4,San Marino,"5,090",90,1.80%,265.80
...,...,...,...,...,...
175,Burundi,"4,905",8,0.20%,0.07
176,Vietnam,"8,791",53,0.60%,0.05
177,Tanzania,509,21,4.10%,0.04
178,Laos,"1,963",3,0.20%,0.04


In [70]:
fig5 = px.bar(df_new[:10], x="País", y="Mortes por cem mil hab.", color="País", title="Os 10 países com mais mortes por cem mil hab.")
fig5.show()

In [74]:
fig6 = px.bar(df_new[:10].sort_values(by="Taxa de Fatalidade", ascending=False), x="País", y="Taxa de Fatalidade", color="País", title="Os 10 países com maior taxa de fatalidade em percentual.")
fig6.show()